In [1]:
import pandas as pd
import numpy as np
import warnings; warnings.simplefilter('ignore')
from google.cloud import bigquery

In [27]:
#connect to bigquery
client = bigquery.Client()

#run query and return dataframe
def run_query(query):
    query_job = client.query(query)
    query_result = query_job.result()
    return query_result.to_dataframe().fillna(np.nan)

pd.options.display.float_format = '{:.3f}'.format

In [31]:
sql_query = f"""
WITH proc AS -- get medicare covid vax procedure claims
    (SELECT claim_number, patient_key, procedure, procedure_type
    FROM `w2ohcdata.drg.submit_procedure`
    WHERE procedure IN('91300', '0001A', '0002A', '91301', '0011A', '0012A', '91302', '0021A', '0022A', 
    '91303', '0031A', '91304', '0041A', '0042A', 'M0201')
    AND par_dt > '2020-10-01'),

    pt AS -- get patient info
    (SELECT claim_number as claim_num_pt, patient_key as pt_key, member_adr_zip, patient_birth_year
    FROM `w2ohcdata.drg.submit_patient`
    WHERE par_dt > '2020-10-01'),

    proc_pt AS -- join procedure + patient on claim_number
    (SELECT *
    FROM proc
    JOIN pt ON proc.claim_number = pt.claim_num_pt),
    
    payer AS -- get payer info
    (SELECT claim_number as claim_num_payer, type_coverage, min_service_from_header
    FROM `w2ohcdata.drg.submit_payer`
    WHERE par_dt > '2020-10-01'),
    
    proc_pt_payer AS -- join procedure_patient + payer on claim_num
    (SELECT * 
    FROM proc_pt
    JOIN payer ON proc_pt.claim_number = payer.claim_num_payer),
    
    coverage AS -- insurance type reference
    (SELECT *
    FROM `w2ohcwork.rwd_references.type_coverage_payment`)

 -- join everything with insurance type
SELECT claim_number, patient_key, procedure, procedure_type, member_adr_zip,
    patient_birth_year, type_coverage, description, insurance_group, min_service_from_header 
FROM proc_pt_payer
JOIN coverage ON proc_pt_payer.type_coverage = coverage.type_coverage_payment
"""

In [33]:
df = run_query(sql_query)
df.head()

,claim_number,patient_key,procedure,procedure_type,member_adr_zip,patient_birth_year,type_coverage,description,insurance_group,min_service_from_header
0,vul_c25424b9-f286-48fd-a70c-b81eae687712,162147178771393196,0011A,CPT,322,1945.000,CH,Champus,VA / Other,2021-01-11
1,344659000000029012,145634713145531454,0001A,CPT,554,2005.000,M,Family or Friends,VA / Other,2021-06-23
2,alb_2128342332,149617144261273516,0012A,CPT,730,1971.000,12,Preferred Provider Organization (PPO),Commercial,2021-04-01
3,342315000000572173,144642953657192662,0002A,CPT,019,1952.000,A,Self-pay,VA / Other,2021-04-26
4,341600000000215506,159600709841481823,0001A,CPT,019,1980.000,O,Other,VA / Other,2021-04-10


In [ ]:
cpt_info = {
    '91300': 'Pfizer-Biontech Covid-19 Vaccine',
    '0001A': 'Pfizer-Biontech Covid-19 Vaccine Administration – First Dose',
    '0002A': 'Pfizer-Biontech Covid-19 Vaccine Administration – Second Dose',
    '91301': 'Moderna Covid-19 Vaccine',
    '0011A': 'Moderna Covid-19 Vaccine Administration – First Dose',
    '0012A': 'Moderna Covid-19 Vaccine Administration – Second Dose',
    '91302': 'AstraZeneca Covid-19 Vaccine',
    '0021A': 'AstraZeneca Covid-19 Vaccine Administration – First Dose',
    '0022A': 'AstraZeneca Covid-19 Vaccine Administration – Second Dose',
    '91303': 'Janssen (Johnson & Johnson) Covid-19 Vaccine',
    '0031A': 'Janssen (Johnson & Johnson) Covid-19 Vaccine Administration',
    '91304': 'Novavax Covid-19 Vaccine',
    '0041A': 'Novavax Covid-19 Vaccine Administration – First Dose',
    '0042A': 'Novavax Covid-19 Vaccine Administration – Second Dose',
    'M0201': 'Covid-19 vaccine administration inside a patient\'s home'
}
df['procedure_description'] = df['procedure'].map(cpt_info)

In [36]:
df = df[['claim_number', 'patient_key', 'procedure', 'procedure_description', 'procedure_type', 'member_adr_zip',
         'patient_birth_year', 'type_coverage', 'description', 'insurance_group', 'min_service_from_header']]
df.head()

,claim_number,patient_key,procedure,procedure_description,procedure_type,member_adr_zip,patient_birth_year,type_coverage,description,insurance_group,min_service_from_header
0,vul_c25424b9-f286-48fd-a70c-b81eae687712,162147178771393196,0011A,Moderna Covid-19 Vaccine Administration – Firs...,CPT,322,1945.000,CH,Champus,VA / Other,2021-01-11
1,344659000000029012,145634713145531454,0001A,Pfizer-Biontech Covid-19 Vaccine Administratio...,CPT,554,2005.000,M,Family or Friends,VA / Other,2021-06-23
2,alb_2128342332,149617144261273516,0012A,Moderna Covid-19 Vaccine Administration – Seco...,CPT,730,1971.000,12,Preferred Provider Organization (PPO),Commercial,2021-04-01
3,342315000000572173,144642953657192662,0002A,Pfizer-Biontech Covid-19 Vaccine Administratio...,CPT,019,1952.000,A,Self-pay,VA / Other,2021-04-26
4,341600000000215506,159600709841481823,0001A,Pfizer-Biontech Covid-19 Vaccine Administratio...,CPT,019,1980.000,O,Other,VA / Other,2021-04-10


In [37]:
len(df)

16441787

In [45]:
len(df.patient_key.unique()) # this seems kinda low

6357981

In [38]:
df.dtypes

claim_number                object
patient_key                  int64
procedure                   object
procedure_description       object
procedure_type              object
member_adr_zip              object
patient_birth_year         float64
type_coverage               object
description                 object
insurance_group             object
min_service_from_header     object
dtype: object

In [40]:
df.isna().any()

claim_number               False
patient_key                False
procedure                  False
procedure_description      False
procedure_type             False
member_adr_zip              True
patient_birth_year          True
type_coverage              False
description                False
insurance_group            False
min_service_from_header    False
dtype: bool

In [41]:
df.procedure_description.value_counts()

Pfizer-Biontech Covid-19 Vaccine Administration – First Dose     3875540
Pfizer-Biontech Covid-19 Vaccine Administration – Second Dose    3445686
Moderna Covid-19 Vaccine Administration – First Dose             3070841
Moderna Covid-19 Vaccine Administration – Second Dose            2579565
Pfizer-Biontech Covid-19 Vaccine                                 1446650
Moderna Covid-19 Vaccine                                         1324122
Janssen (Johnson & Johnson) Covid-19 Vaccine Administration       507654
Janssen (Johnson & Johnson) Covid-19 Vaccine                      191554
AstraZeneca Covid-19 Vaccine Administration – First Dose              81
AstraZeneca Covid-19 Vaccine                                          55
Covid-19 vaccine administration inside a patient's home               33
Novavax Covid-19 Vaccine Administration – First Dose                   4
AstraZeneca Covid-19 Vaccine Administration – Second Dose              2
Name: procedure_description, dtype: int64

In [42]:
df.insurance_group.value_counts()

Commercial    12414636
Medicare       2830549
Medicaid        897491
VA / Other      299111
Name: insurance_group, dtype: int64

In [43]:
df.min_service_from_header = pd.to_datetime(df.min_service_from_header)
df.min_service_from_header.describe()

count                16441787
unique                    219
top       2021-04-08 00:00:00
freq                   221496
first     2020-11-04 00:00:00
last      2021-07-01 00:00:00
Name: min_service_from_header, dtype: object

In [44]:
#df.patient_birth_year = df.patient_birth_year.astype(np.int64)
df.patient_birth_year.describe()

count   16203395.000
mean        1963.631
std           20.032
min         1931.000
25%         1948.000
50%         1959.000
75%         1979.000
max         2021.000
Name: patient_birth_year, dtype: float64